## Dataset and approach:
Data is from Kaggle competiotion [Home Credit Default Risk](https://www.kaggle.com/c/home-credit-default-risk). 

I implement an automated feature engineering approach with an open-source library [Featuretools](https://www.featuretools.com/). 


In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
import featuretools as ft

In [3]:
# Read in the datasets  
app_train = pd.read_csv('../data/application_train.csv', sep=',')
app_test = pd.read_csv('../data/application_test.csv')
bureau = pd.read_csv('../data/bureau.csv')
bureau_balance = pd.read_csv('../data/bureau_balance.csv')
cash = pd.read_csv('../data/POS_CASH_balance.csv')
credit = pd.read_csv('../data/credit_card_balance.csv')
previous = pd.read_csv('../data/previous_application.csv')
installments = pd.read_csv('../data/installments_payments.csv')

![](../images/home_credit_data.png)

In [4]:
datasets_list = [app_train, app_test, bureau, bureau_balance, cash, credit, previous, installments]

app_train.name = 'app_train'
app_test.name = 'app_test'
bureau.name = 'bureau'
bureau_balance.name = 'bureau_balance'
cash.name = 'cash'
credit.name = 'credit'
previous.name = 'previous'
installments.name = 'installments'

In [5]:
for ds in datasets_list:
    print('{}\t - \t{} rows'.format(ds.name , ds.iloc[:, 0].count()))

app_train	 - 	307511 rows
app_test	 - 	48744 rows
bureau	 - 	1716428 rows
bureau_balance	 - 	27299925 rows
cash	 - 	10001358 rows
credit	 - 	3840312 rows
previous	 - 	1670214 rows
installments	 - 	13605401 rows


In [6]:
# replace the anomalous values
for ds in datasets_list:
    ds.replace({365243: np.nan}, inplace=True)

In [7]:
# Join train and test set to make sure, that the same feature are created for each set. 
# Later it will be separated.

In [8]:
app_test['TARGET'] = np.nan
app = app_train.append(app_test, ignore_index=True)

### Featuretools

In [9]:
# Entity set to keep track of all the data
es = ft.EntitySet(id = 'clients')

#### Variable Types

In [10]:
import featuretools.variable_types as vtypes

In [11]:
app_types = {}

In [12]:
# Boolean variables

for col in app.columns:
    if (app[col].nunique() == 2) and (app[col].dtype == float):
        app_types[col] = vtypes.Boolean
        
del app_types['TARGET']

print('Number of Boolean variables: {}'.format(len(app_types)))

Number of Boolean variables: 32


In [13]:
# Ordinal variables
app_types['REGION_RATING_CLIENT'] = vtypes.Ordinal
app_types['REGION_RATING_CLIENT_W_CITY'] = vtypes.Ordinal
app_types['HOUR_APPR_PROCESS_START'] = vtypes.Ordinal

In [14]:
previous_types = {}

for col in previous.columns:
    if ( previous[col].nunique() == 2) and (previous[col].dtype == float):
        previous_types[col] = vtypes.Boolean
        
print('Number of Boolean variables: {}'.format(len(previous_types)))

Number of Boolean variables: 2


Drop `SK_ID_CURR` in `installments`, `credit`, `cash` because I will link to these dataset through `previous` and `SK_ID_PREV`.

To avoid `featuretools` to create useless statistical aggregations of ids.

In [15]:
installments = installments.drop(columns = ['SK_ID_CURR'])
credit = credit.drop(columns = ['SK_ID_CURR'])
cash = cash.drop(columns = ['SK_ID_CURR'])

In [16]:
# Add Entities to EntitySet

es = es.entity_from_dataframe(entity_id = 'app', dataframe = app, index = 'SK_ID_CURR', variable_types=app_types)
es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau, index = 'SK_ID_BUREAU')
es = es.entity_from_dataframe(entity_id = 'previous', dataframe = previous, index = 'SK_ID_PREV', variable_types= previous_types )


In [ ]:
# Entities without unique index. We need to add.
es = es.entity_from_dataframe(entity_id = 'bureau_balance', dataframe = bureau_balance, 
                             make_index = True, index = 'bureaubalance_index')

es = es.entity_from_dataframe(entity_id = 'cash', dataframe = cash, 
                             make_index = True, index = 'cash_index')

es = es.entity_from_dataframe(entity_id = 'installments', dataframe = installments,
                             make_index = True, index = 'installments_index')

es = es.entity_from_dataframe(entity_id = 'credit', dataframe = credit,
                             make_index = True, index = 'credit_index')

In [127]:
# Define relationship
r_app_bureau = ft.Relationship(es['app']['SK_ID_CURR'], es['bureau']['SK_ID_CURR'])

r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])
r_app_previous = ft.Relationship(es['app']['SK_ID_CURR'], es['previous']['SK_ID_CURR'])
r_previous_cash = ft.Relationship(es['previous']['SK_ID_PREV'], es['cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['previous']['SK_ID_PREV'], es['installments']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['previous']['SK_ID_PREV'], es['credit']['SK_ID_PREV'])

In [128]:
# Add relationships to EntitySet

In [129]:
es = es.add_relationships([r_app_bureau])
es = es.add_relationships([r_bureau_balance,  r_previous_cash, r_previous_installments, r_previous_credit])


In [130]:
es

Entityset: clients
  Entities:
    app [Rows: 356255, Columns: 122]
    bureau [Rows: 343286, Columns: 17]
  Relationships:
    bureau.SK_ID_CURR -> app.SK_ID_CURR

#### Feature primitives

In [41]:
primitives = ft.list_primitives()

In [49]:
primitives.head(5)

,name,type,description
0,max,aggregation,Finds the maximum non-null value of a numeric ...
1,num_unique,aggregation,Returns the number of unique categorical varia...
2,last,aggregation,Returns the last value.
3,min,aggregation,Finds the minimum non-null value of a numeric ...
4,time_since_last,aggregation,Time since last related instance.


In [137]:
# Define default premitives
agg_primitives = ["max"]
trans_primitives = ['percentile']
#agg_primitives = ["sum", "max", "min", "mean", "count", "percent_true", "num_unique", "mode"]
#trans_primitives = ['percentile', 'and']
#Deep Feature Synthesis for app

feature_names = ft.dfs(entityset = es, target_entity = 'app', 
                        trans_primitives = trans_primitives,
                        agg_primitives = agg_primitives, 
                        where_primitives = [], seed_features = [],
                        max_depth = 2, n_jobs = 1, verbose = 1,
                        features_only = True)


Built 227 features


In [138]:
feature_names[-5:]

[<Feature: PERCENTILE(MAX(bureau.AMT_CREDIT_SUM_DEBT))>,
 <Feature: PERCENTILE(MAX(bureau.AMT_CREDIT_SUM_LIMIT))>,
 <Feature: PERCENTILE(MAX(bureau.AMT_CREDIT_SUM_OVERDUE))>,
 <Feature: PERCENTILE(MAX(bureau.DAYS_CREDIT_UPDATE))>,
 <Feature: PERCENTILE(MAX(bureau.AMT_ANNUITY))>]

In [133]:
ft.save_features(feature_names, '../input/features.txt')

#### Run full Deep Feature Synthesis

### Note!: 
Code below is to compute in a straight forward way a matrix with all defined features.  
Due to constrains of my local machine, computation will be parallelled with Dask and run on AWS EC2.

In [134]:
import sys
import psutil
print('Total size of entityset: {:.2f} gb.'. format(sys.getsizeof(es)/ 1e9 ))

print('Total number of cpus detected: {} CPUs.'.format( psutil.cpu_count()) )
print('Total size of system memory: {:.2f} gb RAM.'.format( psutil.virtual_memory().total / 1e9) )

Total size of entityset: 0.77 gb.
Total number of cpus detected: 4 CPUs.
Total size of system memory: 4.19 gb RAM.


In [149]:
# start = time.time()

# feature_matrix, features_names = ft.dfs(entityset= es, target_entity='app', 
#                                         agg_primitives=agg_primitives,
#                                         trans_primitives = trans_primitives, 
#                                         where_primitives = [], seed_features = [],
#                                         max_depth = 2, n_jobs = 1, verbose = 1,
#                                         features_only = False)

# end = time.time()

# print('Computation time: {}'.format(end - start))

In [ ]:
feature_matrix.reset_index(inplace = True)
feature_matrix.to_csv('../data/feature_matrix.csv', index=False)